
# Similar Questions Retrieval

This notebook is inspired by the [similar search example of Sentence-Transformers](https://www.sbert.net/examples/applications/semantic-search/README.html#similar-questions-retrieval), and adapted to support [Raft ANN](https://github.com/rapidsai/raft) algorithm.

The model was pre-trained on the [Natural Questions dataset](https://ai.google.com/research/NaturalQuestions). It consists of about 100k real Google search queries, together with an annotated passage from Wikipedia that provides the answer. It is an example of an asymmetric search task. As corpus, we use the smaller [Simple English Wikipedia](http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz) so that it fits easily into memory.

In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch
import pylibraft
from pylibraft.neighbors import ivf_flat, ivf_pq
pylibraft.config.set_output_as(lambda device_ndarray: device_ndarray.copy_to_host())

if not torch.cuda.is_available():
  print("Warning: No GPU found. Please add GPU to your notebook")


# We use the Bi-Encoder to encode all passages, so that we can use it with semantic search
model_name = 'nq-distilbert-base-v1'
bi_encoder = SentenceTransformer(model_name)

# As dataset, we use Simple English Wikipedia. Compared to the full English wikipedia, it has only
# about 170k articles. We split these articles into paragraphs and encode them with the bi-encoder

wikipedia_filepath = 'data/simplewiki-2020-11-01.jsonl.gz'

if not os.path.exists(wikipedia_filepath):
    util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

passages = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())
        for paragraph in data['paragraphs']:
            # We encode the passages as [title, text]
            passages.append([data['title'], paragraph])

# If you like, you can also limit the number of passages you want to use
print("Passages:", len(passages))

# To speed things up, pre-computed embeddings are downloaded.
# The provided file encoded the passages with the model 'nq-distilbert-base-v1'
if model_name == 'nq-distilbert-base-v1':
    embeddings_filepath = 'simplewiki-2020-11-01-nq-distilbert-base-v1.pt'
    if not os.path.exists(embeddings_filepath):
        util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01-nq-distilbert-base-v1.pt', embeddings_filepath)

    corpus_embeddings = torch.load(embeddings_filepath)
    corpus_embeddings = corpus_embeddings.float()  # Convert embedding file to float
    if torch.cuda.is_available():
        corpus_embeddings = corpus_embeddings.to('cuda')
else:  # Here, we compute the corpus_embeddings from scratch (which can take a while depending on the GPU)
    corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

# Vector Search using RAPIDS Raft
Now that our embeddings are ready to be indexed and that the model has been loaded, we can use RAPIDS Raft ANN to do our vector search.

This is done in two step: First we build the index, then we search it.
With pylibraft all you need is those four Python lines:

In [ ]:
%%time
params = ivf_pq.IndexParams(n_lists=150, pq_dim=96)
pq_index = ivf_pq.build(params, corpus_embeddings)
search_params = ivf_pq.SearchParams()

def search_raft_pq(query, top_k = 5):
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)

    hits = ivf_pq.search(search_params, pq_index, question_embedding[None], top_k)

    # Output of top-k hits
    print("Input question:", query)
    for k in range(top_k):
        print("\t{:.3f}\t{}".format(hits[0][0, k], passages[hits[1][0, k]]))

For IVF-PQ we want to reduce the memory footprint while keeping a good accuracy.

In [ ]:
pq_index_mem = pq_index.pq_dim * pq_index.size * pq_index.pq_bits
print("IVF-PQ memory footprint: {:.1f} MB".format(pq_index_mem / 2**20))
original_mem = corpus_embeddings.shape[0] * corpus_embeddings.shape[1] * 4
print("Original dataset: {:.1f} MB".format(original_mem / 2**20))


print("Memory saved: {:.1f}%".format(100 * (1 - pq_index_mem / original_mem)))

In [ ]:
%%time
search_raft_pq(query="Who was Grace Hopper?")

In [ ]:
%%time
search_raft_pq(query="Who was Alan Turing?")

In [ ]:
%%time
search_raft_pq(query = "What is creating tides?")

In [ ]:
%%time
params = ivf_flat.IndexParams(n_lists=150)
flat_index = ivf_flat.build(params, corpus_embeddings)
search_params = ivf_flat.SearchParams()

def search_raft_flat(query, top_k = 5):
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    
    start_time = time.time()
    hits = ivf_flat.search(search_params, flat_index, question_embedding[None], top_k)
    end_time = time.time()

    # Output of top-k hits
    print("Input question:", query)
    print("Results (after {:.3f} seconds):".format(end_time - start_time))
    for k in range(top_k):
        print("\t{:.3f}\t{}".format(hits[0][0, k], passages[hits[1][0, k]]))

In [ ]:
%%time
search_raft_flat(query="Who was Grace Hopper?")

In [ ]:
%%time
search_raft_flat(query="Who was Alan Turing?")

In [ ]:
%%time
search_raft_flat(query = "What is creating tides?")